In [1]:
# Sarah loves music
###
from carabao.neurotron import Pulse
from carabao.util import repr
from numpy import array
from ypstruct import struct

#===================================================================================
# class: Terminal
#===================================================================================

class Terminal:
    """
    class Terminal: to model a McCulloch-Pitts-type synapse terminal
    >>> w = [1,1,0,1,1,1,0,1,0,1]   # binary weights  
    >>> theta = 6                   # spiking threshold
    >>> excite = Terminal(w,theta,None,'excite')
    """
    
    def __init__(self,W,theta,select=None,log=None):
        def matrix(X):
            X = array(X)
            return X if len(X.shape) > 1 else array([X])

        self.select = select        # select unit
        self.W = matrix(W)
        self.theta = theta          # spiking threshold
        self.log = log              # log string
        if log is not None:
            print(self)

    def empower(self,V):            # determine empowerment
        return self.W * array(V)

    def spike(self,E):              # spike function
        S = array([sum(E[k]) for k in range(0,E.shape[0])])
        return (S >= self.theta)*1
        
    def __repr__(self):
        head = "(" if self.log is None else "('%s'," % self.log
        par = head + repr(self.W) + "," + "%g)"%self.theta;
        return "Terminal" + par

#===================================================================================
# class: Neurotron
#===================================================================================

class Core:
    """
    class Core: implements a Neurotron core functionality
    >>> core = Core(k,excite,predict,depress)
    """
    def __init__(self,k,excite,depress,predict,log=None):
        self.k = k
        self.excite  = Terminal(excite.w[k],excite.theta,'excite')
        self.depress = Terminal(depress.w[k],depress.theta,'depress')
        self.predict = Terminal(predict.W[k],predict.theta,'predict')

    def feed(self,f,c,V):
        e = self.excite.empower(f)
        u = self.excite.spike(e)
        print("e:",repr(e), "u:",u)

        v = self.depress.empower(c)
        d = self.depress.spike(v) 
        print("v:",repr(v), "d:",d)

        E = self.predict.empower(V)
        s = self.predict.spike(E)
        print("E:",repr(E), "s:",repr(s))
     
#===================================================================================
# helper: set up toy stuff
#===================================================================================

def toy(mode):
    """
    toy(): setup toy stuff
    >>> excite,predict,depress,token = toy('sarah')  # get parameters for 'sarah' app
    """
    def bundle(obj,n):                           # create a bunch of object as a list
        return [obj for k in range(0,n)]
        
    if mode == 'sarah':
        token = {'Sarah':[1,1,0,1,1,1,0,1,0,1],
                 'likes':[0,1,1,1,0,1,1,0,1,1],
                 'music':[1,1,1,0,0,1,0,1,1,1]}

        f1 = token['Sarah']
        f2 = token['likes']
        f3 = token['music']

        e = struct();                            # excitation terminal parameters 
        e.w = [f1,f2,f3]                         # excitation weights  
        e.theta = 6                              # spiking threshold
        
        p = struct()                             # prediction terminal parameters 
        p.W = bundle([[1,0,0],[0,1,1]],3)        # prediction weights           
        p.theta = 2                              # prediction threshold
        
        d = struct()                             # depression terminal parameters 
        d.w = bundle([1,1,0],1)                  # depression weights           
        d.theta = 1                              # depression threshold

        return (e,d,p,token)


In [2]:
# working with terminals

epar,dpar,ppar,token = toy('sarah')
excite  = Terminal(epar.w[0],ppar.theta,None,'excite')
depress = Terminal(dpar.w[0],dpar.theta,None,'depress')
predict = Terminal(ppar.W[0],ppar.theta,None,'predict')

f=[1,1,0,1,1,1,0,1,0,1]
c=[1,1,1];  V = [c,c]

e = excite.empower(f)
u = excite.spike(e)
print("e:",repr(e), "u:",u)

v = depress.empower(c)
d = depress.spike(v) 
print("v:",repr(v), "d:",d)

E = predict.empower(V)
S = predict.spike(E)
print("E:",repr(E), "S:",repr(S))




Terminal('excite',#[1 1 0 1 1 1 0 1 0 1],2)
Terminal('depress',#[1 1 0],1)
Terminal('predict',#[1 0 0; 0 1 1],2)
e: #[1 1 0 1 1 1 0 1 0 1] u: [1]
v: #[1 1 0] d: [1]
E: #[1 0 0; 0 1 1] S: :[0 1]


In [3]:
# working with a Neurotron core

epar,dpar,ppar,token = toy('sarah')
excite  = Terminal(epar.w[0],ppar.theta,'excite')
depress = Terminal(dpar.w[0],dpar.theta,'depress')
predict = Terminal(ppar.W[0],ppar.theta,'predict')

core = Core(0,epar,dpar,ppar,'core0')

f=[1,1,0,1,1,1,0,1,0,1]
c=[1,1,1];  V = [c,c]

e = core.excite.empower(f)
u = core.excite.spike(e)
print("e:",repr(e), "u:",u)

E = core.predict.empower(V)
s = core.predict.spike(E)
print("E:",repr(E), "s:",repr(s))

v = core.depress.empower(c)
d = core.depress.spike(E) 
print("v:",repr(v), "d:",d)


e: #[1 1 0 1 1 1 0 1 0 1] u: [1]
E: #[1 0 0; 0 1 1] s: :[0 1]
v: #[1 1 0] d: [1 1]


In [4]:
# feeding the Neurotron core

excite,predict,depress,token = toy('sarah')
core = Core(0,excite,predict,depress) 

f=[1,1,0,1,1,1,0,1,0,1]
c=[1,1,1]; V = [c,c]

core.feed(f,c,V)

e: #[1 1 0 1 1 1 0 1 0 1] u: [1]
v: #[1 1 0] d: [1]
E: #[1 0 0; 0 1 1] s: :[0 1]


In [5]:
e = core.excite.empower(token['music'])
u = core.excite.spike(e)
(repr(e),repr(u))

('#[1 1 0 0 0 1 0 1 0 1]', ':[0]')

In [6]:
par = toy('sarah')
excite  = Terminal(par[0].w[0],par[0].theta,'excite',log='excite')


Terminal('excite',#[1 1 0 1 1 1 0 1 0 1],6)
